In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.22.0 to work with expworkspace


In [2]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [3]:
# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-EarlySdiabetes-exp")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes_dataset.csv')

Starting experiment: mslearn-EarlySdiabetes-exp
Loading Data...


In [4]:
# Change column names: replace spaces with underscores
diabetes.rename(columns={'sudden weight loss': 'sudden_weight_loss', 'Genital thrush': 'Genital_thrush', 
                         'visual blurring': 'visual_blurring', 'delayed healing': 'delayed_healing', 
                         'partial paresis': 'partial_paresis', 'muscle stiffness': 'muscle_stiffness'}, inplace=True)

In [5]:
# making capitalizing the fisrt letter of each head column 
diabetes.columns = map(lambda x: str(x).capitalize(), diabetes.columns)

#changing data values to integer
change = {'Age':'Age', 'Gender':{'Male':1, 'Female':0}, 'Polyuria':{'Yes':1, 'No':0},       
        'Polydipsia':{'Yes':1, 'No':0}, 'Sudden_weight_loss':{'Yes':1, 'No':0},
          'Weakness':{'Yes':1, 'No':0}, 'Polyphagia':{'Yes':1, 'No':0},
         'Genital_thrush':{'Yes':1, 'No':0}, 'Visual_blurring':{'Yes':1, 'No':0},
          'Itching':{'Yes':1, 'No':0}, 'Irritability':{'Yes':1, 'No':0}, 
          'Delayed_healing':{'Yes':1, 'No':0},'Partial_paresis':{'Yes':1, 'No':0}, 
          'Muscle_stiffness':{'Yes':1, 'No':0}, 'Alopecia':{'Yes':1, 'No':0}, 
          'Obesity':{'Yes':1, 'No':0}, 'Class':{'Positive':1, 'Negative':0}}


for k, v in change.items():
    diabetes[k] = diabetes[k].replace(v)


In [6]:
# Split the test and train data
X = diabetes.drop(columns='Class')
y = diabetes['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [7]:
# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

Training a decision tree model


In [8]:
# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

Accuracy: 0.967948717948718
AUC: 0.9679135209334249


In [9]:
# Save the trained model
model_file = 'Es_diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

In [10]:
# Complete the run
run.complete()

In [11]:
# Register the model
run.register_model(model_path='outputs/Es_diabetes_model.pkl', model_name='Es_diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')


Model trained and registered.


In [14]:
# Check the model version
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Es_diabetes_model version: 1
	 Training context : Inline Training
	 AUC : 0.9679135209334249
	 Accuracy : 0.967948717948718


EarlyStage_diabetes_model version: 1
	 Training context : Inline Training
	 AUC : 0.9679135209334249
	 Accuracy : 0.967948717948718




In [15]:
model = ws.models['Es_diabetes_model']
print(model.name, 'version', model.version)

Es_diabetes_model version 1


In [16]:
import os

folder_name = 'Es_diabetes_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_Esdiabetes.py")

Es_diabetes_service folder created.


In [21]:
%%writefile $script_file
# Score file created
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('Es_diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['Negative', 'Positive']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing ./Es_diabetes_service/score_Esdiabetes.py


In [19]:
# create and save environment
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"Es_diabetes_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./Es_diabetes_service/Es_diabetes_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [22]:
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

In [23]:
from azureml.core.compute import ComputeTarget, AksCompute

In [27]:
# AKs cluster creation
cluster_name = 'aksCluster' 
compute_config = AksCompute.provisioning_configuration(location='eastus',cluster_purpose='DevTest')
Dev_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
Dev_cluster.wait_for_completion(show_output=True)

Creating....................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [28]:
# Service deployment configuration
from azureml.core.webservice import AksWebservice

deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

In [32]:
# Model deployment

from azureml.core.model import Model

model = ws.models['Es_diabetes_model']
service = Model.deploy(workspace=ws,
                       name = 'es-diabetes-service',
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deploy_config,
                       deployment_target = Dev_cluster)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................................................................................
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [33]:
print(service.state)

Healthy


In [34]:
print(service.get_logs())

2021-02-25T13:40:53,205875842+00:00 - gunicorn/run 
2021-02-25T13:40:53,206780741+00:00 - iot-server/run 
2021-02-25T13:40:53,207188840+00:00 - rsyslog/run 
2021-02-25T13:40:53,209386137+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b824bcb98517d791c41923f24d65461/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [35]:
for webservice_name in ws.webservices:
    print(webservice_name)


es-diabetes-service


In [63]:
import json

x_new = X_test[0:10].values.tolist()
print ('Patient: {}'.format(x_new[0]))

Patient: [67, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]


In [64]:
# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})
# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [67, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1] Positive
Patient [47, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0] Positive
Patient [43, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1] Positive
Patient [26, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Negative
Patient [60, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0] Negative
Patient [58, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0] Positive
Patient [35, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Positive
Patient [56, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0] Positive
Patient [38, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1] Positive
Patient [31, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0] Positive


In [76]:
endpoint = service.scoring_uri
print(endpoint)

primary_key, secondary_key = service.get_keys()
print(primary_key)

http://52.224.129.216:80/api/v1/service/es-diabetes-service/score
cLqJ64Ia1h3smXkaVt3BeIT3UigJw9Q0


In [103]:
import requests
import json

x_new = X_test[0:10].values.tolist()

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + primary_key}

# Call the service
response = requests.post(url = endpoint,
                         data = input_json,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print(x_new[i], predictions[i])

[67, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1] Positive
[47, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0] Positive
[43, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1] Positive
[26, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Negative
[60, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0] Negative
[58, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0] Positive
[35, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Positive
[56, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0] Positive
[38, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1] Positive
[31, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0] Positive
